# 解决手写数字识别的问题

### 数据准备

在开始之前，我们将需要训练的数据进行打包，并使用加载器以方便排列组合。

按照实训所要求的，数据分为训练数据集和比赛数据集，其中训练数据集用于构建训练数据，比赛数据集用于构建比赛提交。

在开始之前，我们应当从一部分训练数据集中分出来一部分作为测试数据集，目前尚未实现这样的功能，考虑在写数据集优化的时候添加重映射来实现这个功能。在计划中采用前四分之三的部分用于训练数据集，后四分之一用于测试数据集，测试数据集不参与模型优化。

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch import optim
from torch import nn
import torch

# 可视化日志
writer = SummaryWriter(".\logs")

# 比赛数据集
test = pd.read_csv("./data/test.csv")
test_data = test/255
test_data = test_data.values.reshape(-1, 28, 28, 1)


class TrainDataset(Dataset):
    # 创建训练数据集
    def __init__(self, path: str, train: bool = True, transform=None) -> None:
        data_csv = pd.read_csv(path)
        self.transform = transform
        self.train = train
        # 提取训练集
        self.label = data_csv['label']
        data = data_csv.drop(labels=["label"], axis=1)
        # 归一化
        data = data/255
        # 图像重塑
        self.data = data.values.reshape(-1, 28, 28, 1)
        # 目标值向量化
        self.target = nn.functional.one_hot(torch.tensor(self.label))/1
        # 尾处理
        self.total = len(self.label)
        self.cut = int(self.total/4)
        super().__init__()

    def __len__(self) -> int:
        if not self.train:
            return self.cut
        return self.total-self.cut

    def __getitem__(self, index: int):
        if not self.train:
            index = self.total-index-1
        image = self.data[index]
        label = self.label[index]
        target = self.target[index]
        if self.transform:
            image = self.transform(image)
        return {'image': image,
                'label': label,
                'target': target}


# 数据集加载器
test_loader = DataLoader(
    TrainDataset(
        path="./data/train.csv",
        train=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((28, 28)),
        ]),
    ),
    batch_size=8*8,
    shuffle=False,
    num_workers=0,
    drop_last=False
)

train_loader = DataLoader(
    TrainDataset(
        path="./data/train.csv",
        train=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((28, 28)),
        ]),
    ),
    batch_size=8*8,
    shuffle=False,
    num_workers=0,
    drop_last=False
)


### 模型训练前准备

模型参数需要存储在本地，一些训练的参数也要准备，其中最重要的就是模型的定义，我们采用两层卷积网络和两层全连接层，损失函数使用CrossEntropy，模型优化器选用SGD。

在后期调试的时候应引用tensorboard。

In [22]:
# 参数设置
learning_rate = 0.001            # 学习率
module_file_name = "./module.pth" # 模型存储

# 创建模型
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(32,64,5,padding=2),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            nn.Flatten(),
            nn.Linear(7*7*64,1000),
            nn.ReLU(),
            nn.Linear(1000,10),
            nn.Softmax(1),
        )

    def forward(self, x):
        x=x.to(torch.float32)
        x = self.model(x)
        return x
    
# GPU加速
t_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(t_device)
# 损失函数
loss = nn.CrossEntropyLoss().to(t_device)
# 优化器
opti = optim.SGD(model.parameters(), lr=learning_rate)
# 尝试加载本地模型
model.load_state_dict(torch.load(module_file_name))

<All keys matched successfully>

### 训练阶段

在一切准备完成之后就要对模型进行训练了，我们将数据以每组64个输入模型，同时计算64个网络来加速训练过程。

In [13]:
for epoch in range(10):
    step = 0
    step_test = 0
    loss_epoch = 0.0
    loss_epoch_test = 0.0
    total_accuracy = 0.0
    
    # 训练中
    model.train()
    for i_batch,batch_data in enumerate(train_loader):
        image = batch_data['image'].to(t_device)
        target = batch_data['target'].to(t_device)
        if step % 10 == 0:
            print("|", end="")
        opti.zero_grad() # 优化器置零
        output = model(image)
        result_loss = loss(output, target)
        result_loss.backward()
        opti.step()
        loss_epoch += result_loss
        step += 1
        
    # 模型验证过程
    model.eval()
    with torch.no_grad():
        for i_batch,batch_data in enumerate(test_loader):
            image = batch_data['image'].to(t_device)
            target = batch_data['target'].to(t_device)
            label = batch_data['label'].to(t_device)
            if step_test % 10 == 0:
                print("¦", end="")

            output = model(image)
            result_loss = loss(output, target)

            accuracy = (output.argmax(1) == label).sum()
            total_accuracy += accuracy
            loss_epoch_test += result_loss
            step_test += 1

    print("\nepoch:", epoch+1, "loss(total,train):%.2f" % loss_epoch,
          "loss(total,test):%.2f" % loss_epoch_test,
          "accuacry(test):%.2f" % (total_accuracy/(len(test_loader)*64)))
    writer.add_scalar("Ayala-loss", loss_epoch, epoch)
    writer.add_scalar("Ayala-test-loss", loss_epoch_test, epoch)
    writer.add_scalar("Ayala-test-accuracy",
                      total_accuracy/(len(test_loader)*64), epoch)

torch.save(model.state_dict(), module_file_name)

||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 1 loss(total,train):726.05 loss(total,test):243.88 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 2 loss(total,train):725.91 loss(total,test):243.86 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 3 loss(total,train):725.84 loss(total,test):243.85 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 4 loss(total,train):725.92 loss(total,test):243.85 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 5 loss(total,train):725.96 loss(total,test):243.84 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 6 loss(total,train):725.92 loss(total,test):243.84 accuacry(test):0.98
||||||||||||||||||||||||||||||||||||||||||||||||||¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦¦
epoch: 7 loss(total,train):725.99 loss(total,test):243.8

### 对比赛提供的数据集构建提交

在上一步我们初步得到了一个98%准确度的模型，我们将使用这个模型并对比赛进行提交。

In [27]:
import csv

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28)),
])

idx = []
ans = []

for index, image in enumerate(test_data):
    image = transform(image)
    image = torch.reshape(image, (1, 1, 28, 28))
    model.eval()
    with torch.no_grad():
        output = model(image)
        target = int(output.argmax(1))
        idx.append(index+1)
        ans.append(target)
        
frame = pd.DataFrame({'ImageId':idx,'Label':ans})
frame.to_csv("./submission.csv",index=False,sep=',')

-----
✨差不多结束了，你可以在Kaggle上提交了🎉